<a href="https://colab.research.google.com/github/15483114/trabalho-nlp-fiap/blob/main/Template_Trabalho_Final_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Participantes (RM - NOME):***<br>
347291 - Henrique Diniz<br>
347017 - Isabella Kuo<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>

###**Criar um classificador de chamados aplicando técnicas de PLN**
---

A **QuantumFinance** tem um canal de atendimento via chat e precisar classificar os assuntos dos atendimentos para melhorar as tratativas dos chamados dos clientes. O canal recebe textos abertos dos clientes relatando o problema e/ou dúvida e depois é direcionado para algum uma área especialista no assunto para uma melhor tratativa.​

Crie um modelo classificador de assuntos aplicando técnicas de PLN, que consiga classificar através de um texto o assunto conforme disponível na base de dados [1] para treinamento e validação do modelo seu modelo.​

O modelo precisar atingir um score na **métrica F1 Score superior a 75%**. Utilize o dataset [1] para treinar e testar o modelo, separe o dataset em duas amostras (75% para treinamento e 25% para teste com o randon_state igual a 42).​

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

**Composição da nota:​**

**50%** - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, organização do pipeline, etc.)​

**50%** - Baseado na performance (score) obtida com a amostra de teste no pipeline do modelo campeão (validar com  a Métrica F1 Score). **Separar o pipeline completo do modelo campeão conforme template.​**

O trabalho poderá ser feito em grupo de até 4 pessoas (mesmo grupo do Startup One).

**[1] = ​https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv**

In [ ]:
# CARREGANDO O DATA FRAME
import pandas as pd
df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')

# Façam o download do arquivo e utilizem localmente durante os testes

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21072 entries, 0 to 21071
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_reclamacao         21072 non-null  int64 
 1   data_abertura         21072 non-null  object
 2   categoria             21072 non-null  object
 3   descricao_reclamacao  21072 non-null  object
dtypes: int64(1), object(3)
memory usage: 658.6+ KB


Bom desenvolvimento!

In [ ]:
df

,id_reclamacao,data_abertura,categoria,descricao_reclamacao,tudo
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,"Bom dia, meu nome é xxxx xxxx e agradeço se vo...","Hipotecas / Empréstimos Bom dia, meu nome é xx..."
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...,Cartão de crédito / Cartão pré-pago Atualizei ...
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,O cartão Chase foi relatado em xx/xx/2019. No ...,Cartão de crédito / Cartão pré-pago O cartão C...
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,"Em xx/xx/2018, enquanto tentava reservar um ti...",Cartão de crédito / Cartão pré-pago Em xx/xx/2...
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,"Meu neto me dê cheque por {$ 1600,00} Eu depos...",Serviços de conta bancária Meu neto me dê cheq...
...,...,...,...,...,...
21067,3094545,2018-12-07T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Depois de ser um cliente de cartão de persegui...,Cartão de crédito / Cartão pré-pago Depois de ...
21068,3091984,2018-12-05T12:00:00-05:00,Roubo / Relatório de disputa,"Na quarta -feira, xx/xx/xxxx, liguei para o Ch...","Roubo / Relatório de disputa Na quarta -feira,..."
21069,3133355,2019-01-25T12:00:00-05:00,Roubo / Relatório de disputa,Não estou familiarizado com o XXXX Pay e não e...,Roubo / Relatório de disputa Não estou familia...
21070,3110963,2018-12-27T12:00:00-05:00,Outros,Eu tive crédito impecável por 30 anos. Eu tive...,Outros Eu tive crédito impecável por 30 anos. ...


Pré processamento

In [ ]:
df['categoria'].value_counts()

Serviços de conta bancária             5161
Cartão de crédito / Cartão pré-pago    5006
Roubo / Relatório de disputa           4822
Hipotecas / Empréstimos                3850
Outros                                 2233
Name: categoria, dtype: int64

In [ ]:
# removendo pontuação e acentuação e colocando tudo em letra minuscula
import re

corpus = []
for x in df['descricao_reclamacao']:
  review = re.sub('[^a-zA-Z]', ' ', x)
  review = review.lower()
  corpus.append(review)
df['descricao_reclamacao'] = corpus

In [ ]:
df

,id_reclamacao,data_abertura,categoria,descricao_reclamacao
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,bom dia meu nome xxxx xxxx e agrade o se vo...
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,atualizei meu cart o xxxx xxxx em xx xx e...
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,o cart o chase foi relatado em xx xx no ...
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,em xx xx enquanto tentava reservar um ti...
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,meu neto me d cheque por eu depos...
...,...,...,...,...
21067,3094545,2018-12-07T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,depois de ser um cliente de cart o de persegui...
21068,3091984,2018-12-05T12:00:00-05:00,Roubo / Relatório de disputa,na quarta feira xx xx xxxx liguei para o ch...
21069,3133355,2019-01-25T12:00:00-05:00,Roubo / Relatório de disputa,n o estou familiarizado com o xxxx pay e n o e...
21070,3110963,2018-12-27T12:00:00-05:00,Outros,eu tive cr dito impec vel por anos eu tive...


Separando em treino e teste e escolhendo ngrams

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')

stops = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Criar uma instância do vetorizador TF-IDF
# vectorizer = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
vectorizer = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops)
vectorizer.fit(df["descricao_reclamacao"])
# Vetorizar os dados de texto
X_1 = vectorizer.fit_transform(df["descricao_reclamacao"])

In [ ]:
# Criar uma instância do vetorizador TF-IDF
# vectorizer = TfidfVectorizer(ngram_range=(2,2), use_idf=True)
vectorizer = TfidfVectorizer(ngram_range=(2,2), use_idf=True, stop_words=stops)
vectorizer.fit(df["descricao_reclamacao"])
# Vetorizar os dados de texto
X_2 = vectorizer.fit_transform(df["descricao_reclamacao"])

In [ ]:
# Criar uma instância do vetorizador TF-IDF
# vectorizer = TfidfVectorizer(ngram_range=(3,3), use_idf=True)
vectorizer = TfidfVectorizer(ngram_range=(3,3), use_idf=True, stop_words=stops)
vectorizer.fit(df["descricao_reclamacao"])
# Vetorizar os dados de texto
X_3 = vectorizer.fit_transform(df["descricao_reclamacao"])

In [ ]:
X = X_1
# X = X_2
# X = X_3

In [ ]:
from sklearn.preprocessing import LabelEncoder

y = df['categoria']
# Criar uma instância do LabelEncoder
label_encoder = LabelEncoder()

# Aplicar o LabelEncoder aos rótulos
y_encoded = label_encoder.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_treino, X_teste, y_treino, y_teste = train_test_split(X,y_encoded, test_size = 0.25)

Como o problema é multiclasse utilizaremos de outras varições do f1 score para validar o modelo



In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score

# Create a Naive Bayes classifier
classifier = MultinomialNB()

# Train the classifier
classifier.fit(X_treino, y_treino)

# Make predictions on the test set
y_pred = classifier.predict(X_teste)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_teste, y_pred)
print("Accuracy:", accuracy)
from sklearn.metrics import f1_score

# Macro-Averaged F1 Score
f1_macro = f1_score(y_teste, y_pred, average='macro')

# Micro-Averaged F1 Score
f1_micro = f1_score(y_teste, y_pred, average='micro')

# Weighted-Averaged F1 Score
f1_weighted = f1_score(y_teste, y_pred, average='weighted')

print("Macro-Averaged F1 Score:", f1_macro)
print("Micro-Averaged F1 Score:", f1_micro)
print("Weighted-Averaged F1 Score:", f1_weighted)

Accuracy: 0.739749430523918
Macro-Averaged F1 Score: 0.6374554093673994
Micro-Averaged F1 Score: 0.739749430523918
Weighted-Averaged F1 Score: 0.703329638184693


Unigrama:

Accuracy: 0.7348139711465452

Macro-Averaged F1 Score: 0.6263281248733904

Micro-Averaged F1 Score: 0.7348139711465452

Weighted-Averaged F1 Score: 0.6989145954572505

Bigrama:

Accuracy: 0.7260820045558086

Macro-Averaged F1 Score: 0.6225399037594219

Micro-Averaged F1 Score: 0.7260820045558086

Weighted-Averaged F1 Score: 0.6893620400596979

Trigrama:

Accuracy: 0.6909643128321944

Macro-Averaged F1 Score: 0.5906447182564049

Micro-Averaged F1 Score: 0.6909643128321944

Weighted-Averaged F1 Score: 0.6523435504092788


Unigrama sem stop words:

Accuracy: 0.739749430523918

Macro-Averaged F1 Score: 0.6374554093673994

Micro-Averaged F1 Score: 0.739749430523918

Weighted-Averaged F1 Score: 0.703329638184693

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
classifier.fit(X_treino, y_treino)

# Make predictions on the test set
y_pred = classifier.predict(X_teste)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_teste, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.821753986332574


In [ ]:
from sklearn.metrics import f1_score

# Macro-Averaged F1 Score
f1_macro = f1_score(y_teste, y_pred, average='macro')

# Micro-Averaged F1 Score
f1_micro = f1_score(y_teste, y_pred, average='micro')

# Weighted-Averaged F1 Score
f1_weighted = f1_score(y_teste, y_pred, average='weighted')

print("Macro-Averaged F1 Score:", f1_macro)
print("Micro-Averaged F1 Score:", f1_micro)
print("Weighted-Averaged F1 Score:", f1_weighted)

Macro-Averaged F1 Score: 0.7834035787753042
Micro-Averaged F1 Score: 0.821753986332574
Weighted-Averaged F1 Score: 0.8117158662240792


Unigrama:

Accuracy: 0.816628701594533

Macro-Averaged F1 Score: 0.7690315302323467

Micro-Averaged F1 Score: 0.813591495823842

Weighted-Averaged F1 Score: 0.8013434075942348

Bigrama:

Accuracy: 0.781700835231587

Macro-Averaged F1 Score: 0.7380965640294812

Micro-Averaged F1 Score: 0.781700835231587

Weighted-Averaged F1 Score: 0.7691803633474319

Trigrama:

Accuracy: 0.7126044039483675

Macro-Averaged F1 Score: 0.6537402160176755

Micro-Averaged F1 Score: 0.7126044039483677

Weighted-Averaged F1 Score: 0.6914457429509279


Unigrama sem stop words:

Accuracy: 0.821753986332574

Macro-Averaged F1 Score: 0.7834035787753042

Micro-Averaged F1 Score: 0.821753986332574

Weighted-Averaged F1 Score: 0.8117158662240792


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

svm_model = SVC()

# Train the classifier
svm_model.fit(X_treino, y_treino)

# Make predictions on the test set
y_pred = svm_model.predict(X_teste)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_teste, y_pred)
print("Accuracy:", accuracy)
from sklearn.metrics import f1_score

# Macro-Averaged F1 Score
f1_macro = f1_score(y_teste, y_pred, average='macro')

# Micro-Averaged F1 Score
f1_micro = f1_score(y_teste, y_pred, average='micro')

# Weighted-Averaged F1 Score
f1_weighted = f1_score(y_teste, y_pred, average='weighted')

print("Macro-Averaged F1 Score:", f1_macro)
print("Micro-Averaged F1 Score:", f1_micro)
print("Weighted-Averaged F1 Score:", f1_weighted)

Accuracy: 0.89958238420653
Macro-Averaged F1 Score: 0.8974390696997606
Micro-Averaged F1 Score: 0.89958238420653
Weighted-Averaged F1 Score: 0.8993241668338668


Unigrama:
Accuracy: 0.89958238420653
Macro-Averaged F1 Score: 0.8974390696997606
Micro-Averaged F1 Score: 0.89958238420653
Weighted-Averaged F1 Score: 0.8993241668338668


Trigrama:

Accuracy: 0.7050113895216401
Macro-Averaged F1 Score: 0.6294979998062805
Micro-Averaged F1 Score: 0.7050113895216401
Weighted-Averaged F1 Score: 0.6786483822848587

Unigrama sem stop words:

Accuracy: 0.89958238420653
Macro-Averaged F1 Score: 0.8974390696997606
Micro-Averaged F1 Score: 0.89958238420653
Weighted-Averaged F1 Score: 0.8993241668338668

####**Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.

Carregando dados

In [ ]:
# CARREGANDO O DATA FRAME
import pandas as pd
df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')

# Façam o download do arquivo e utilizem localmente durante os testes

Pré processamento

In [ ]:
# removendo pontuação e acentuação e colocando tudo em letra minuscula
import re

corpus = []
for x in df['descricao_reclamacao']:
  review = re.sub('[^a-zA-Z]', ' ', x)
  review = review.lower()
  corpus.append(review)
df['descricao_reclamacao'] = corpus

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')

stops = nltk.corpus.stopwords.words('portuguese')

vectorizer = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops)
vectorizer.fit(df["descricao_reclamacao"])

X = vectorizer.fit_transform(df["descricao_reclamacao"])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.preprocessing import LabelEncoder

y = df['categoria']
# Criar uma instância do LabelEncoder
label_encoder = LabelEncoder()

# Aplicar o LabelEncoder aos rótulos
y_encoded = label_encoder.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_treino, X_teste, y_treino, y_teste = train_test_split(X,y_encoded, test_size = 0.25)

Modelo

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

svm_model = SVC()

# Train the classifier
svm_model.fit(X_treino, y_treino)

# Make predictions on the test set
y_pred = svm_model.predict(X_teste)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_teste, y_pred)
print("Accuracy:", accuracy)
from sklearn.metrics import f1_score


# Macro-Averaged F1 Score
f1_macro = f1_score(y_teste, y_pred, average='macro')

# Micro-Averaged F1 Score
f1_micro = f1_score(y_teste, y_pred, average='micro')

# Weighted-Averaged F1 Score
f1_weighted = f1_score(y_teste, y_pred, average='weighted')

print("Macro-Averaged F1 Score:", f1_macro)
print("Micro-Averaged F1 Score:", f1_micro)
print("Weighted-Averaged F1 Score:", f1_weighted)

Accuracy: 0.8954062262718299
Macro-Averaged F1 Score: 0.8919647543274261
Micro-Averaged F1 Score: 0.8954062262718299
Weighted-Averaged F1 Score: 0.8950937375750277
